In [1]:
%matplotlib inline
from qiskit import IBMQ, QuantumCircuit, QuantumRegister, ClassicalRegister, Aer

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-afrl', group='air-force-lab', project='educational')>,
 <IBMQBackend('ibmq_20_tokyo') from IBMQ(hub='ibm-q-afrl', group='air-force-lab', project='educational')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-afrl', group='air-force-lab', project='educational')>]

In [2]:
backend = provider.get_backend('ibmq_poughkeepsie')

In [4]:
backend.properties()

BackendProperties(backend_name='ibmq_poughkeepsie', backend_version='1.2.0', gates=[Gate(gate='id', name='id_0', parameters=[Nduv(date=datetime.datetime(2019, 8, 13, 23, 19, 7, tzinfo=tzutc()), name='gate_error', unit='', value=0.0010091566201823532), Nduv(date=datetime.datetime(2019, 8, 14, 5, 17, 56, 520433, tzinfo=tzutc()), name='gate_length', unit='ns', value=113.77777777777777)], qubits=[0]), Gate(gate='u1', name='u1_0', parameters=[Nduv(date=datetime.datetime(2019, 8, 13, 23, 19, 7, tzinfo=tzutc()), name='gate_error', unit='', value=0.0), Nduv(date=datetime.datetime(2019, 8, 14, 5, 17, 56, 520471, tzinfo=tzutc()), name='gate_length', unit='ns', value=0.0)], qubits=[0]), Gate(gate='u2', name='u2_0', parameters=[Nduv(date=datetime.datetime(2019, 8, 13, 23, 19, 7, tzinfo=tzutc()), name='gate_error', unit='', value=0.0010091566201823532), Nduv(date=datetime.datetime(2019, 8, 14, 5, 17, 56, 520505, tzinfo=tzutc()), name='gate_length', unit='ns', value=103.1111111111111)], qubits=[0]),

In [1]:
# Import the quantum volume sub-module from Ignis
import qiskit.ignis.verification.quantum_volume as qv

# Also import Aer's noise model functionality to allow us to compare actual performance
# to that expected from device 1Q and 2Q error rates
from qiskit.providers.aer.noise.device.models import basic_device_noise_model

import matplotlib.pyplot as plt

# Import the quantum volume sub-module from Ignis
import qiskit.ignis.verification.quantum_volume as qv

# Also import Aer's noise model functionality to allow us to compare actual performance
# to that expected from device 1Q and 2Q error rates
from qiskit.providers.aer.noise.device.models import basic_device_noise_model

In [8]:
help(qv.qv_circuits)

Help on function qv_circuits in module qiskit.ignis.verification.quantum_volume.circuits:

qv_circuits(qubit_lists=None, ntrials=1, qr=None, cr=None)
    Return a list of square quantum volume circuits (depth=width)
    
    The qubit_lists is specified as a list of qubit lists. For each
    set of qubits, circuits the depth as the number of qubits in the list
    are generated
    
    Args:
        qubit_lists: list of list of qubits to apply qv circuits to. Assume
        the list is ordered in increasing number of qubits
        ntrials: number of random iterations
        qr: quantum register to act on (if None one is created)
        cr: classical register to measure to (if None one is created)
    
    Returns:
        qv_circs: list of lists of circuits for the qv sequences
        (separate list for each trial)
        qv_circs_nomeas: same as above with no measurements for the ideal
        simulation

